In [21]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage
import math
import io

In [22]:
#Get google credentials downloaded by terraform
file_path = "Terraform/gcp_credentials.json"
credentials = service_account.Credentials.from_service_account_file(file_path)

In [23]:
#Add your project name here 
client = bigquery.Client(credentials=credentials, project="ncaa-project-455709")
storage_client = storage.Client()

In [24]:
# Should be - bigquery-public-data, ncaa_basketball
project_id = "bigquery-public-data"
dataset_id = "ncaa_basketball"

In [25]:
dataset_ref = client.dataset(dataset_id, project=project_id)
tables = list(client.list_tables(dataset_ref))

In [26]:
# Should be - ncaa-bucket-dump
bucket_name = "ncaa-bucket-dump"
bucket = storage_client.get_bucket(bucket_name)

In [27]:
tables

In [34]:
table_name = "team_colors"
query_job = client.query(f"""
                                  SELECT *
                                  FROM `{project_id}.{dataset_id}.{table_name}`
                                  """)
df = query_job.to_dataframe()



In [ ]:
df.head()

,market,id,code_ncaa,color
0,Milwaukee,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,797,#000000
1,Colorado,9fccbf28-2858-4263-821c-fdefb3c7efa3,157,#000000
2,Northeastern,93df9b18-e9fc-42a7-bb45-a736c203b4dc,500,#000000
3,Georgia Southern,6b955b96-b736-475e-bffd-e4acf2054169,253,#000066
4,Richmond,9b66e1e0-aace-4671-9be2-54c8acf5ecfc,575,#000066


In [36]:
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [37]:
df.head()

,id,code_ncaa,color
0,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,797,#000000
1,9fccbf28-2858-4263-821c-fdefb3c7efa3,157,#000000
2,93df9b18-e9fc-42a7-bb45-a736c203b4dc,500,#000000
3,6b955b96-b736-475e-bffd-e4acf2054169,253,#000066
4,9b66e1e0-aace-4671-9be2-54c8acf5ecfc,575,#000066
